In [1]:
inputs = list(map(int, open('Day23.txt').read()))

In [2]:
class Item(int):
    def __new__(cls, value):
        new_int = int.__new__(cls, value)
        new_int.prev = x.next = None
        return new_int
        
class Deque:
    def __init__(self, values=None):
        self.current = None
        self.items = {}
        self.extend(values or ())
        
    def extend(self, values):
        first = self.current
        last = first.prev if first else None
        for value in values:
            item = Item(value)
            self.items[item] = item
            if not last:
                first = item
            else:
                last.next = item
                item.prev = last
            last = item
        if first:
            first.prev = last
        if last:
            last.next = first
        if not self.current:
            self.current = first
    
    def pop_next(self):
        item = self.current.next
        self.current.next = item.next
        item.next.prev = self.current
        item.prev = item.next = None
        del self.items[item]
        return item
    
    def pop_many(self, count):
        return [self.pop_next() for i in range(count)]
    
    def insert_next(self, item):
        item.next = self.current.next
        item.prev = self.current
        self.current.next = item
        item.next.prev = item
        self.items[item] = item
        
    def insert_many_after(self, items, item):
        current = self.current
        self.current = self.items[item]
        for item in reversed(items):
            self.insert_next(item)
        self.current = current
        
    def return_to_start(self):
        self.current = self.items[1]
        return self.current
        
    def forward(self):
        self.current = self.current.next
        return self.current
        
    def __iter__(self):
        item = self.current
        yield item
        item = item.next
        while item is not self.current:
            yield item
            item = item.next

In [3]:
from tqdm.notebook import trange

def play(inputs, turns=100, maximum=None):
    minimum = min(inputs)
    cups = Deque(inputs)
    if maximum:
        cups.extend(range(max(inputs) + 1, maximum + 1))
    maximum = maximum or max(inputs)
    for turn in trange(turns):
        pick = cups.pop_many(3)
        dest = cups.current - 1
        if dest < minimum:
            dest = maximum
        while dest in pick:
            dest -= 1
            if dest < minimum:
                dest = maximum
        cups.insert_many_after(pick, dest)
        cups.forward()
    cups.return_to_start()
    return cups

In [4]:
%%time
cups = play(inputs, turns=100)
int(''.join(map(str, cups))[1:])


CPU times: user 34.4 ms, sys: 1.27 ms, total: 35.7 ms
Wall time: 33 ms


97632548

In [5]:
%%time
from operator import mul
cups = play(inputs, turns=10_000_000, maximum=1_000_000)
mul(cups.current.next, cups.current.next.next)


CPU times: user 1min 3s, sys: 780 ms, total: 1min 3s
Wall time: 1min 3s


412990492266